# Music Database
We all know what music is right? How would we model a database to hold the information?


For the simplicity's sake, let's ignore collaborations and compilations and model things like this...

ARTIST has one to many TRACKs
ALBUM has one to many TRACKS
ARTIST has one to many ALBUMs
TRACK has one ARTIST
TRACK has one ALBUM

So we need to create 3 tables
ARTIST
ALBUM
TRACK

but first, lets connect to a database

In [ ]:
import sqlalchemy as sa

DB_NAME="music.db"
engine = sa.create_engine(f"sqlite:///{DB_NAME}")
meta = sa.MetaData()

# Define Tables with SqlAlchemy

In [ ]:
# artist table
tbl_artist = sa.Table('artist', meta,
                  sa.Column('id',
                            sa.Integer, 
                            primary_key=True, 
                            autoincrement=True),
                 sa.Column('name', 
                           sa.String, 
                           unique=True, 
                           nullable=False))
# album table
tbl_album = sa.Table('album', meta, 
                    sa.Column('id',
                             sa.Integer,
                             primary_key=True,
                             autoincrement=True),
                    sa.Column('name', 
                              sa.String, 
                             nullable=False),
                    sa.Column('year', 
                              sa.String), 
                    sa.Column('artist_id', 
                              sa.Integer,
                             sa.ForeignKey('artist.id')))
# track table
tbl_track = sa.Table('track', meta,
                    sa.Column('id', 
                              sa.Integer, 
                              primary_key=True,
                              autoincrement=True),
                    sa.Column('path', 
                             sa.String,
                             unique=True,
                             nullable=False),
                    sa.Column('artist_id', 
                              sa.Integer, 
                              sa.ForeignKey('artist.id'),
                              nullable=False),
                    sa.Column('album_id', 
                              sa.Integer,
                              sa.ForeignKey('album.id')),
                     sa.Column('name', 
                               sa.String, 
                               nullable=False),
                    sa.Column('track_number', 
                              sa.Integer),
                    sa.Column('genre', sa.String))
meta.drop_all(bind=engine)
meta.create_all(bind=engine)

# read in music data prepared by the tags.py file on my collection which took about 10 minutes


In [ ]:
import json
music_json = None
with open('music.json', 'r') as f:
    music_json = json.loads(f.read())

In [ ]:
music_json[0]

# need to get data from music_json into the artist, album and track tables

In [ ]:
for i, track_json in enumerate(music_json[0:999]):
    with engine.begin() as connection:
        # insert artist if not exists
        tag_artist_name = track_json['tag_data'].get('artist', [''])[0]
    
        # check for existence with query
        artist_exists = tbl_artist.select().where(
            tbl_artist.c.name==tag_artist_name
        )
        db_result = connection.execute(artist_exists).first()
        if db_result:
            artist_id = db_result['id']
        else:
            ins_artist = tbl_artist.insert().values(name=tag_artist_name)
            db_result = connection.execute(ins_artist)
            artist_id = db_result.lastrowid

        # insert album if not exists
        tag_album_name = track_json['tag_data'].get('album', [''])[0]
        tag_album_year = track_json['tag_data'].get('date', [''])[0]
        album_exists = tbl_album.select().where(sa.and_(
            tbl_album.c.artist_id==artist_id,
            tbl_album.c.name==tag_album_name,
            tbl_album.c.year==tag_album_year
        ))
        db_result = connection.execute(album_exists).first()
        if db_result:
            album_id = db_result['id']
        else:
            ins_album = tbl_album.insert().values(
                name=tag_album_name,
                year=tag_album_year,
                artist_id=artist_id)
            db_result = connection.execute(ins_album)
            album_id = db_result.lastrowid
        
        # insert track if not exists
        tag_track_name = track_json['tag_data']['title'][0]
        tag_track_path = track_json['file']
        tag_track_track_number = track_json['tag_data'].get('tracknumber', [''])[0]
        tag_track_genre = track_json['tag_data'].get('genre', [''])[0]
        track_exists = tbl_track.select().where(
            tbl_track.c.path==tag_track_path
        )
        db_result = connection.execute(track_exists).first()
        if not db_result:
            ins_track = tbl_track.insert().values(
                artist_id=artist_id,
                album_id=album_id,
                path=tag_track_path,
                name=tag_track_name,
                track_number=tag_track_track_number,
                genre=tag_track_genre            
            )
            connection.execute(ins_track)
print('done importing')

In [ ]:
engine.execute("select * from artist").fetchall()